## Welcome to version 3: 25.July

This version is currently wokring on doin the split train and test!

TODO LIST:
- Create a trainable linear map for feature extraction
- Propose using (sentnce,img) as opposed to (sentence,img,img)

Complete:
- Negative images downloaded
- Check error in accessing images in the UK 

==

- Split the data in train, dev, and test (NOW)
- Create labels with the dataframe (NOW)
- Remove articles from sentences

#### Imports

In [12]:
import os
import requests
import pandas as pd
from PIL import Image
import io
import numpy as np
from torch import nn

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from sklearn.decomposition import PCA

from lambeq.backend.grammar import Ty, Box 
from lambeq import BobcatParser, AtomicType, Sim14Ansatz, RemoveCupsRewriter

import matplotlib.pyplot as plt

print("All libraries are successfully imported!")

All libraries are successfully imported!


#### Dataset and Image pre-processing

Dataset extraction

In [13]:
# Read the CSV file
file_path = os.path.join(os.getcwd(), "database.csv")
df = pd.read_csv(file_path)

# Extract the sentence
sentence = df['sentence']
image_pos = df['pos_url']
image_neg = df['neg_url']

df = pd.DataFrame({
    'sentence': sentence,
    'image_pos': image_pos,
    'image_neg': image_neg
})

df.head()

,sentence,image_pos,image_neg
0,man stand behind the wall,https://previews.123rf.com/images/ryanking999/ryanking9991702/ryanking999170200409/72735211-apuesto-hombre-de-pie-con-pared-de-ladrillo-asi%C3%A1tico.jpg,https://previews.123rf.com/images/ryanking999/ryanking9991703/ryanking999170300165/72844322-hombre-sentarse-y-sentirse-deprimido-con-pared-de-ladrillo-asi%C3%A1tico.jpg
1,The woman is sitting against the wall.,http://www.colourbox.com/preview/5020075-522756-young-woman-sitting-on-the-wooden-floor-against-white-wall.jpg,https://st4.depositphotos.com/1462687/20482/i/1600/depositphotos_204822478-stock-photo-pretty-woman-casual-clothes-sit.jpg
2,Man doing exercise sits in front of a wall,https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/workouts/2017/09/wall-sit-1506433841.gif,https://previews.123rf.com/images/ryanking999/ryanking9991702/ryanking999170200409/72735211-apuesto-hombre-de-pie-con-pared-de-ladrillo-asi%C3%A1tico.jpg
3,A man is standing on a wall.,https://previews.123rf.com/images/deagreez/deagreez2001/deagreez200104071/139177244-full-size-photo-of-gorgeous-brutal-man-stand-isolated-over-grey-wall-background-touch-hand-his-hairc.jpg,https://thumbs.dreamstime.com/z/%E5%9C%A8%E7%8E%B0-%E4%BC%81%E4%B8%9A%E5%A4%A7%E5%8E%A6%E5%89%8D%E9%9D%A2%E7%9A%84-%E4%BA%BA%E7%AB%8B%E5%9C%BA-32285543.jpg
4,A man is standing on a wall.,https://previews.123rf.com/images/deagreez/deagreez2001/deagreez200104071/139177244-full-size-photo-of-gorgeous-brutal-man-stand-isolated-over-grey-wall-background-touch-hand-his-hairc.jpg,https://image.shutterstock.com/z/stock-photo-1297043299.jpg


Download images functions

In [14]:
# function to download an image
def Download_Images(url):
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))
    return img

# function to pre-process an image
def process_image(img, n_components, base_model):
    # Downlaod image
    img = Download_Images(img)

    image = img.resize((224, 224))

    if image.mode != 'RGB':
        # print("Image is greyscale")
        image = image.convert("RGB")

    # Convert the image to an array
    img_array = np.array(image)
    # print("(1) Image shape: ", img_array.shape)

    # Expand dimensions to match the shape (1, 224, 224, 3)
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the input
    img_array = preprocess_input(img_array)

    # print(f"Raw image data: {img_array}")
    # print(f"(2) Image shape: {img_array.shape}")
    # return(img_array)

    # ========================================
    #           Extract features 
    # ========================================
    features = base_model.predict(img_array)

    # # Define the PCA object
    pca = PCA(n_components=n_components)

    features = features.reshape((features.shape[0], -1))

    # print(f"Image shape for PCA: {features.shape}")
    replicated_feature_vector = np.tile(features, (n_components, 1))
    pca.fit(replicated_feature_vector)
    reduced_vector = pca.transform(features)

    # # print("\nReduced shape:", reduced_vector.shape)
    reshaped_tensor = reduced_vector.reshape(-1)

    return reshaped_tensor



<span style="background-color: red"> ERROR: </span>
- Images are deleted from the dataframe with error when downloading, this is done manuallly from implementation.ipynb

In [15]:
# Read the CSV file
# completed above

pos_indexToRemove = []
neg_indexToRemove = []

# Function to download an image
def download_image(index, url, indexToRemove):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        img = Image.open(io.BytesIO(response.content))
        # img.save(save_path)
        # print(f"Downloaded and saved {url} to {save_path}")
    except requests.exceptions.Timeout:
        print(f"Index: {index} || Timeout: {url} took too long to download.")
        indexToRemove.append(index)
    except Exception as e:
        print(f"Index: {index} || Failed to download {url}: {e}" )
        indexToRemove.append(index)

# Iterate over the URLs in the DataFrame
print("Image Positive")
for index, row in df.iterrows():
    url_pos = row['image_pos']  # Replace 'image_url' with the column name in your CSV
    download_image(index, url_pos, pos_indexToRemove)
print(f"Indicies to remove for positive images: {pos_indexToRemove}")

print("\nImage Negative ")
for index, row in df.iterrows():
    url_neg = row['image_neg']  # Replace 'image_url' with the column name in your CSV
    download_image(index, url_neg, neg_indexToRemove)
print(f"Indicies for negative: {neg_indexToRemove}")

print("Done: All images downloaded.")

Image Positive
Index: 5 || Failed to download https://comps.canstockphoto.at/unten-sitzen-wand-junges-schauen-stock-foto_csp18140520.jpg: HTTPSConnectionPool(host='comps.canstockphoto.at', port=443): Max retries exceeded with url: /unten-sitzen-wand-junges-schauen-stock-foto_csp18140520.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x160738690>: Failed to resolve 'comps.canstockphoto.at' ([Errno 8] nodename nor servname provided, or not known)"))
Index: 13 || Failed to download https://media.istockphoto.com/photos/man-and-woman-lie-on-a-deserted-sea-beach-and-hug-summer-vacation-picture-id1224829740: 400 Client Error: Bad Request for url: https://media.istockphoto.com/photos/man-and-woman-lie-on-a-deserted-sea-beach-and-hug-summer-vacation-picture-id1224829740
Index: 16 || Failed to download https://cdn.knewsodisha.com/wp-content/uploads/2020/07/RUN-1.jpg: 530 Server Error:  for url: https://cdn.knewsodisha.com/wp-content/uploads/2020/07/RUN-1.jpg
In

Indicied of images that failed to download for positive and negative

In [18]:
print(f" {len(pos_indexToRemove)} Positive indicies failed to download: {pos_indexToRemove}")
print(f" {len(neg_indexToRemove)} Negative indicies failed to download: {neg_indexToRemove}")

 21 Positive indicies failed to download: [5, 13, 16, 23, 30, 55, 69, 76, 88, 104, 118, 122, 125, 132, 137, 146, 147, 153, 154, 166, 181]
 35 Negative indicies failed to download: [10, 11, 13, 15, 24, 25, 32, 42, 44, 45, 48, 55, 56, 59, 64, 67, 74, 83, 84, 86, 89, 92, 115, 117, 118, 126, 131, 142, 143, 151, 158, 166, 177, 182, 188]


Drop images that are unable to download for positive and negative

In [19]:
clean_df = df.copy()
merged_toRemove = list(set(neg_indexToRemove) | set(pos_indexToRemove))
merged_toRemove.sort()
print(merged_toRemove)

for index, row in clean_df.iterrows():
    if index in merged_toRemove:
        # print("Removing index: ", index)
        clean_df = clean_df.drop(index = index)

print(f"Length of original dataframe = ", len(df))
print(f"Length of cleaned dataframe = ", len(clean_df))

[5, 10, 11, 13, 15, 16, 23, 24, 25, 30, 32, 42, 44, 45, 48, 55, 56, 59, 64, 67, 69, 74, 76, 83, 84, 86, 88, 89, 92, 104, 115, 117, 118, 122, 125, 126, 131, 132, 137, 142, 143, 146, 147, 151, 153, 154, 158, 166, 177, 181, 182, 188]
Length of original dataframe =  191
Length of cleaned dataframe =  139


In [20]:
clean_df.head(10)

,sentence,image_pos,image_neg
0,man stand behind the wall,https://previews.123rf.com/images/ryanking999/ryanking9991702/ryanking999170200409/72735211-apuesto-hombre-de-pie-con-pared-de-ladrillo-asi%C3%A1tico.jpg,https://previews.123rf.com/images/ryanking999/ryanking9991703/ryanking999170300165/72844322-hombre-sentarse-y-sentirse-deprimido-con-pared-de-ladrillo-asi%C3%A1tico.jpg
1,The woman is sitting against the wall.,http://www.colourbox.com/preview/5020075-522756-young-woman-sitting-on-the-wooden-floor-against-white-wall.jpg,https://st4.depositphotos.com/1462687/20482/i/1600/depositphotos_204822478-stock-photo-pretty-woman-casual-clothes-sit.jpg
2,Man doing exercise sits in front of a wall,https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/workouts/2017/09/wall-sit-1506433841.gif,https://previews.123rf.com/images/ryanking999/ryanking9991702/ryanking999170200409/72735211-apuesto-hombre-de-pie-con-pared-de-ladrillo-asi%C3%A1tico.jpg
3,A man is standing on a wall.,https://previews.123rf.com/images/deagreez/deagreez2001/deagreez200104071/139177244-full-size-photo-of-gorgeous-brutal-man-stand-isolated-over-grey-wall-background-touch-hand-his-hairc.jpg,https://thumbs.dreamstime.com/z/%E5%9C%A8%E7%8E%B0-%E4%BC%81%E4%B8%9A%E5%A4%A7%E5%8E%A6%E5%89%8D%E9%9D%A2%E7%9A%84-%E4%BA%BA%E7%AB%8B%E5%9C%BA-32285543.jpg
4,A man is standing on a wall.,https://previews.123rf.com/images/deagreez/deagreez2001/deagreez200104071/139177244-full-size-photo-of-gorgeous-brutal-man-stand-isolated-over-grey-wall-background-touch-hand-his-hairc.jpg,https://image.shutterstock.com/z/stock-photo-1297043299.jpg
6,a person sitting on the wall,https://c7.uihere.com/files/281/692/909/woman-female-sit-sitting.jpg,https://c0.anyrgb.com/images/795/432/human-person-kid-hide-and-seek-climbing-caucasian-child-denim-playing-sit.jpg
7,A person sitting against a wall.,https://previews.123rf.com/images/ryanking999/ryanking9991703/ryanking999170300165/72844322-hombre-sentarse-y-sentirse-deprimido-con-pared-de-ladrillo-asi%C3%A1tico.jpg,https://static1.bigstockphoto.com/1/3/2/large1500/231684076.jpg
8,girl standing against wall,https://image.shutterstock.com/z/stock-photo-1297043299.jpg,https://thumbs.dreamstime.com/z/%E5%B0%8F%E4%BA%9A%E6%B4%B2%E5%A5%B3%E5%AD%A9%E7%AB%99%E5%9C%A8%E6%9C%A8%E6%9D%90%E4%B8%8A%EF%BC%8C%E5%9C%A8%E5%85%AC%E5%9B%AD%E6%88%96%E8%8A%B1%E5%9B%AD%E9%87%8C%E7%8E%A9%E8%80%8D%EF%BC%8C%E7%9C%8B%E4%B8%8A%E5%8E%BB%E5%BE%88%E6%9C%89%E8%B6%A3-173974957.jpg
9,A woman sits in a field.,https://previews.123rf.com/images/studio-54/studio-541809/studio-54180900019/108577037-full-length-body-beautiful-girl-sit-on-soccer-green-grass-field-sporty-sexy-woman-with-football-ball.jpg,https://st3.depositphotos.com/4675069/15498/i/1600/depositphotos_154989568-stock-photo-gorgeous-man-and-woman-stand.jpg
12,A person runs on the street,https://i.pinimg.com/originals/27/d0/3c/27d03c3862afb70246d0f6b1534d55cb.jpg,https://c8.alamy.com/comp/2BW2GR8/children-and-toddlers-played-in-the-street-even-a-child-of-this-age-could-be-allowed-to-play-outside-wander-off-and-come-back-with-the-care-of-the-policeman-on-the-beat-2BW2GR8.jpg


Download images officialy 

In [ ]:
print("Download images officially")

model = ResNet50(weights='imagenet', include_top=False)
dim_reduction = 16

# Iterate through each row in the DataFrame
for index, row in clean_df.iterrows():
    print("\nIndex : ", index)

    print(" +ve")
    row["image_pos"] = process_image(row["image_pos"], dim_reduction, model)

    print(" -ve")
    row["image_neg"] = process_image(row["image_neg"], dim_reduction, model)

print("\nDone: positive and ngeative images downloaded successfully")

show head of the clean_df

In [ ]:
make_list = clean_df['image_pos'].apply(lambda x: x.tolist()).tolist()
clean_df['sentence'] = clean_df['sentence'].astype(str)

pd.set_option('display.max_colwidth', None)  # or set it to a specific number of characters
clean_df.head()

In [ ]:
clean_df['label_image1'] = 0
clean_df['label_image2'] = 1
clean_df.head()

#### Quantum Circuit 

Sentence DisCoCat

- Added a prepositional phrase with 1 qubits
- this result in some output of n not sentence why?

In [ ]:
from lambeq import BobcatParser, AtomicType, Sim14Ansatz, RemoveCupsRewriter

remove_cups = RemoveCupsRewriter()

# Parse the sentence and convert it into a string diagram
sentences = clean_df["sentence"]

# # Use BobbcatParser
parser = BobcatParser(verbose='suppress')
raw_sentence_diagrams = [parser.sentence2diagram(sentence) for sentence in sentences]

ansatz_sentence = Sim14Ansatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1, AtomicType.PREPOSITIONAL_PHRASE: 1},n_layers=1)

# print('↓ sample sentence diagram with cups')
# raw_sentence_diagrams[1].draw(figsize=(14, 3), fontsize=12)

train_diagrams_sentence = [remove_cups(sentence_diagram) for sentence_diagram in raw_sentence_diagrams]
train_circuits_sentence = [ansatz_sentence(diagram) for diagram in train_diagrams_sentence]
# print('↓ sample sentence diagram without cups')
# train_diagrams_sentence[1].draw()

In [ ]:
print('↓ sample sentence diagram with cups')
raw_sentence_diagrams[5].draw(figsize=(14, 3), fontsize=12)

print('↓ sample sentence diagram without cups')
train_diagrams_sentence[5].draw()

In [ ]:
print('↓ sample sentence circuit')
train_circuits_sentence[10].draw(figsize=(30, 20), fontsize=12)

Images 

In [ ]:
# Create a new image Ty
image_type = Ty("image")
image_box = Box(name = "IMAGE", dom = Ty(), cod= image_type)
image_box.draw(figsize=(14, 3), fontsize=12)

# Create template Ansatz
ansatz = Sim14Ansatz({image_type: 4}, n_layers=1)
image_circ = ansatz(image_box)
print('↓ template image circuit')
image_circ.draw(figsize=(20, 10), fontsize=12)

# Add data into image circuit POSITIVE
images_pos = clean_df["image_pos"]
train_circuits_imagePos = [image_circ.lambdify(*list(image_circ.free_symbols))(*feature_vec) for feature_vec in images_pos]
print("circuit for positive image: complete")
# train_circuits_imagePos[9].draw(figsize=(20, 10), fontsize=12)

# Add data into image circuit NEGATIVE
images_neg = clean_df["image_neg"]
train_circuits_imageNeg = [image_circ.lambdify(*list(image_circ.free_symbols))(*feature_vec) for feature_vec in images_neg]
print("circuit for negative image: complete")
# train_circuits_imageNeg[9].draw(figsize=(20, 10), fontsize=12)



In [ ]:
train_circuits_imagePos[1].draw(figsize=(20, 10), fontsize=12)
train_circuits_imageNeg[1].draw(figsize=(20, 10), fontsize=12)

merge sentence @ image

In [ ]:
# Create new Type
image_type = Ty("image")
output_type = Ty("output")
comparison_box = Box("COMPARISON", AtomicType.SENTENCE @ image_type @ image_type, output_type)

# diagrams = [(sentence_diagram @ image_type @ image_type) >> comparison_box for sentence_diagram in train_diagrams_sentence]
# diagram = raw_sentence_diagrams[5] @ image_type @ image_type >> comparison_box

# Draws diagram with comparison box (0,1) - Sentence digarm and comparison box is in grammar form
diagrams = []
print("Note: diagrams that cause an error are not added to the list")
print("=======================")
for num in range(len(clean_df)):
    try:
        temp = train_diagrams_sentence[num] @ image_type @ image_type >> comparison_box
        diagrams.append(temp)
    except Exception as e:
        print(f"Error at index {num}: {e}")
print("=======================")

print("diagrams complete")

# diagrams = [(sentence_diagram @ image_type @ image_type) >> comparison_box for sentence_diagram in train_diagrams_sentence]
# print('↓ template merge diagram : (sentence, image1, image2)')
# diagrams[0].draw(figsize=(20, 10), fontsize=12)


In [ ]:
# # Ansatz & Circuit
ansatz = Sim14Ansatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1, AtomicType.PREPOSITIONAL_PHRASE: 1, image_type : 4, output_type : 1},  n_layers=1)
temp_circuit = [ansatz(diagram) for diagram in diagrams]
print("ansatz complete")

In [ ]:
print("template circuit : (sentence, image_type , image_type)")
temp_circuit[0].draw(figsize=(60, 40), fontsize=18, draw_type_labels = False )

add image data

In [ ]:
image_1 = train_circuits_imagePos
image_2 = train_circuits_imageNeg

print(f"Length of circuits =  {len(temp_circuit)}")

# final_circuit = image_1 @ image_2 >> temp_circuit

final_circuit = [(image_1[index] @ image_2[index] >> temp_circuit[index]) for index in range(len(temp_circuit))]

In [ ]:
final_circuit[2].draw(figsize=(30, 15), fontsize=7, draw_type_labels = False )